In [14]:
#c27c7825d1f34093804ceab847772901

import datetime
import requests

# Define your API key and endpoint
api_key = 'c27c7825d1f34093804ceab847772901'
endpoint = 'https://newsapi.org/v2/everything'

# Function to get news articles related to the commodity and filter within the last 24 hours
def fetch_news(query):
    # Calculate the date and time 24 hours ago from now
    yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).isoformat()
    
    params = {
        'q': query,
        'language': 'en',
        'from': yesterday,  # Filter for news published in the last 24 hours
        'sortBy': 'publishedAt',
        'apiKey': api_key
    }
    response = requests.get(endpoint, params=params)
    articles = response.json().get('articles', [])
    return articles

from textblob import TextBlob

# Perform sentiment analysis on the article content or description
def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # returns a polarity score between -1 and 1

# Function to generate buy/sell/hold signal based on article sentiment
def generate_signal(articles, threshold=0.2):
    buy_count = 0
    sell_count = 0
    article_details = []  # To store article details for review

    for article in articles:
        # Get article details
        title = article.get('title')
        description = article.get('description') or article.get('content', "")
        published_at = article.get('publishedAt')
        
        # Perform sentiment analysis
        sentiment_score = analyze_sentiment(description)
        
        # Store article details for DataFrame
        article_details.append({
            "title": title,
            "description": description,
            "published_at": published_at,
            "sentiment_score": sentiment_score
        })

        # Count the article as a buy or sell signal based on sentiment
        if sentiment_score > threshold:
            buy_count += 1
        elif sentiment_score < -threshold:
            sell_count += 1

    # Determine the final signal based on counts
    if buy_count > sell_count:
        signal = "Buy"
    elif sell_count > buy_count:
        signal = "Sell"
    else:
        signal = "Hold"

    # Return the signal and the detailed article information
    return signal, article_details


In [15]:
import pandas as pd

def get_signals_for_commodities():
    commodities = {
        "gold": ["inflation", "interest rates", "gold demand", "gold production"],
        "silver": ["industrial demand", "solar energy", "silver mining", "precious metals"],
        "copper": ["construction", "infrastructure", "China demand", "copper mining"]
    }
    
    all_data = []  # List to store data for the DataFrame

    for commodity, keywords in commodities.items():
        # Fetch news articles related to each keyword in the last 24 hours
        articles = []
        for keyword in keywords:
            articles += fetch_news(f"{commodity} {keyword}")

        # Generate buy/sell/hold signal and get detailed article data
        signal, detailed_articles = generate_signal(articles)
        
        # Append the data to the all_data list
        for article in detailed_articles:
            all_data.append({
                "commodity": commodity.capitalize(),
                "signal": signal,
                "title": article["title"],
                "description": article["description"],
                "published_at": article["published_at"],
                "sentiment_score": article["sentiment_score"]
            })
    
    # Create a DataFrame from all the collected data
    df = pd.DataFrame(all_data)
    return df


In [16]:
# Get real-time signals and detailed article data as a DataFrame
df_signals = get_signals_for_commodities()
print(df_signals)


df_signals.to_csv('news_data_signals.csv', index=False)



   commodity signal                                              title  \
0       Gold    Buy  Trump’s Election: How It Affects The Energy Se...   
1       Gold    Buy  Inside Economics: Why Trump might mean higher ...   
2       Gold    Buy  Inside Economics: Why Trump might mean higher ...   
3       Gold    Buy  What a US Strategic Bitcoin Reserve could look...   
4       Gold    Buy  Taming inflation: Cost of living is the Grinch...   
5       Gold    Buy  BTC hits $82k, Inflation Data this week, PNUT ...   
6       Gold    Buy  Stock market today: Trump rally slows with sto...   
7       Gold    Buy  Biotech-Ville U.S.A.: The Story Behind America...   
8       Gold    Buy  Inside Economics: Why Trump might mean higher ...   
9       Gold    Buy  Inside Economics: Why Trump might mean higher ...   
10      Gold    Buy  'If youve lost your personality, theres no poi...   
11      Gold    Buy  'Stupid Risks That Worked' – Investor Whose Po...   
12      Gold    Buy  Biotech-Ville U.S